In [1]:
import velvetvae as vt

# general packages
import numpy as np
import pandas as pd
import torch
from scipy.sparse import issparse

# velocity packages
import scanpy as sc
import scvelo as scv
import anndata as ann

# plotting packages
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm, trange
from IPython.display import clear_output

# color palette object
from colors import colorpalette as colpal

Global seed set to 0
/camp/home/maizelr/.local/lib/python3.10/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/camp/home/maizelr/.local/lib/python3.10/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [2]:
from benchmarking_functions import *

import velvetvae as vt
import torch

In [4]:
def velvet_pipeline(adata0, name, NC='latent_space', DG='learned'):
    adata = adata0.copy()

    adata.layers['total'] = (
        adata.layers['total'].A if issparse(adata.layers['total']) else adata.layers['total']
    )
    adata.layers['new'] = (
        adata.layers['new'].A if issparse(adata.layers['new']) else adata.layers['new']
    )
    
    vt.pp.neighborhood(adata, n_neighbors=100)
    
    vt.ut.set_seed(0)
    
    vt.md.Velvet.setup_anndata(adata, x_layer='total', n_layer='new', knn_layer='knn_index')

    model = vt.md.Velvet(
        adata,
        n_latent = 50,
        linear_decoder = True,
        neighborhood_space = NC,
        biophysical_model = "full",
        gamma_mode = DG,
        labelling_time = 2.0,
    )

    model.setup_model()
    
    model.train(
        batch_size = adata.shape[0],
        max_epochs = 1000, 
        freeze_vae_after_epochs = 200,
        constrain_vf_after_epochs = 200,
        lr=0.001,
    )
    
    V = model.predict_velocity()
    V = V.A if issparse(V) else V
    V = np.nan_to_num(V, nan=0, neginf=0, posinf=0)
    
    return V

In [ ]:
for name, nc, dg in zip(
    ["velvet_NoC_NoG","velvet_NoC_WiG","velvet_WiC_NoG","velvet_HDC_WiG"],
    ["none","none","latent_space","gene_space"],
    ["fixed","learned","fixed","learned"],
):
    print(f"Running {name}...")
    def func(adata, name):
        return velvet_pipeline(adata, name, NC=nc, DG=dg)
    
    perform_benchmark(
        pipeline_name=name,
        velocity_pipeline=func, 
        output_folder='../../output_data/benchmarking_scores'
    )

Running velvet_NoC_NoG...


  0%|          | 0/6 [00:00<?, ?it/s]

Using ScanPy methods to calculate distances, indices, connectivities, transitions, ... done! 
KNN indices for Velvet stored in .obsm['knn_index'].
Dense similarity transition matrix for Velvet stored in .obsm['Ts'].
INFO     Generating sequential column names                                                  


Multiprocessing is handled by SLURM.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
SLURM auto-requeueing enabled. Setting signal handlers.


Epoch 1000/1000: 100%|██████████| 1000/1000 [01:18<00:00, 12.63it/s, loss=1.17, v_num=1]  

...

Epoch 1000/1000: 100%|██████████| 1000/1000 [05:01<00:00,  3.32it/s, loss=1.05, v_num=1]
computing neighbors
    finished (0:00:04) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
midi_Ne


 83%|████████▎ | 5/6 [17:57<04:37, 277.41s/it]

Using ScanPy methods to calculate distances, indices, connectivities, transitions, ... done! 
KNN indices for Velvet stored in .obsm['knn_index'].
Dense similarity transition matrix for Velvet stored in .obsm['Ts'].
INFO     Generating sequential column names                                                  


Multiprocessing is handled by SLURM.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
SLURM auto-requeueing enabled. Setting signal handlers.


Epoch 855/1000:  85%|████████▌ | 854/1000 [08:44<01:27,  1.66it/s, loss=1.03, v_num=1]    